# 00 — Universe Provider (tickers from package or file)

This notebook explains the new **Universe Provider** concept.

Instead of passing tickers manually every day, you can:
- load a predefined universe shipped with the package (CSV inside the repo/package)
- load your own universe from a local file
- optionally cap the amount of tickers (`--top` / `max_tickers`) to speed up downloads

Key idea:
> Universe Provider returns the list of tickers to analyze, and ensures the benchmark (SPY) is included.

In [ ]:
import pandas as pd
pd.set_option("display.width", 140)
pd.set_option("display.max_columns", 80)

## 1) Load a universe from the package (e.g. `mega`)

In [ ]:
from swing_screener.data.universe import load_universe_from_package, UniverseConfig

tickers = load_universe_from_package("mega", UniverseConfig(max_tickers=50))
tickers[:15], len(tickers)

Notice SPY is included (required for Relative Strength).

In [ ]:
"SPY" in tickers

## 2) Cap the universe size (like CLI `--top`)

In [ ]:
tickers10 = load_universe_from_package("mega", UniverseConfig(max_tickers=10))
tickers10, len(tickers10)

## 3) Load a universe from a local file (`--universe-file`)

Create a small universe file in the notebook (for demo).  
Real usage: you keep a `my_tickers.csv` somewhere and point the CLI to it.

In [ ]:
from pathlib import Path

p = Path("my_tickers.csv")
p.write_text("AAPL\nMSFT\nNVDA\nAMZN\n# comment\nMETA\n", encoding="utf-8")
str(p)

In [ ]:
from swing_screener.data.universe import load_universe_from_file

tickers_file = load_universe_from_file("my_tickers.csv", UniverseConfig(max_tickers=None))
tickers_file

## 4) Fetch data using universe tickers

In [ ]:
from swing_screener.data.market_data import fetch_ohlcv, MarketDataConfig

ohlcv = fetch_ohlcv(tickers10 + ["SPY"], MarketDataConfig(start="2023-01-01"))
ohlcv.head()

If you get rate limits or slow downloads, reduce universe size via `max_tickers`
or use caching in `fetch_ohlcv`.